In [ ]:
import os
import gc
import pickle
import statsmodels.api as sm
import pandas as pd
import numpy as np
import math
import seaborn as sns
from seaborn_analyzer import regplot
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
from datetime import datetime, timedelta
from scipy import stats
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import cross_val_score, validation_curve
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

In [ ]:
def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

#### 実績データ群からDataFrameに変換して.pklファイルとして保存する関数

In [ ]:
def read_file(file_path: str, encoding: str) -> pd.DataFrame:
    """
    指定されたファイルからデータフレームを読み込む関数。
    
    :param file_path: ファイルパス
    :param encoding: ファイルのエンコーディング
    :return: 読み込まれたデータフレーム
    """
    
    # カスタムセパレーターを使用してファイルをDataFrameに読み込み
    df = pd.read_csv(file_path, encoding=encoding, header=None, sep='\s*,\s*', engine='python')

    # 列名を変更
    df.columns = ['time', 'value', 'description', 'number']

    # time列をDatetime型に変換
    df['time'] = pd.to_datetime(df['time'], format='%Y%m%d%H%M')

    # インデックスをtime列に設定
    df.set_index('time', inplace=True)

    return df

def read_folder(folder_path: str, encoding: str) -> pd.DataFrame:
    """
    指定されたフォルダ内のすべてのテキストファイルからデータフレームを読み込む関数。
    
     既に読み込まれている場合は、そのファイルはスキップし、そのファイル名が出力されます。
    
     :param folder_path: フォルダパス
     :param encoding: ファイルのエンコーディング
     :return: 読み込まれたデータフレーム（時系列順）
     """
    
     # 結果用の空のデータフレームを作成
    result = pd.DataFrame()
     
      # フォルダ内のすべてのテキストファイルに対して繰り返し処理する。
    for file_name in sorted(os.listdir(folder_path)):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            if os.path.basename(file_path) in result.index:
                print(f"Skipping {file_name} (already loaded)")
                continue

            df = read_file(file_path, encoding)
            result = pd.concat([result, df])
             
    return result

# 関数を呼び出して結果を表示
df = read_folder('/workspaces/data/flow/data/', 'shift_jis')
df.to_pickle('/workspaces/data/flow/20220518-20230323.pkl')
df.head()

#### .pklファイルから実績元データをロードし、名称振りなおす

In [ ]:
df = pd.read_pickle('/workspaces/data/flow/20220518-20230323.pkl')
df_mapping_table = pd.read_csv('/workspaces/data/flow/対応表.csv',header=None, skiprows=1)
df_mapping_table.columns = ['number', 'wcs', 'description', '', '', '', '', '']
df = df.reset_index().merge(df_mapping_table[['number', 'description']], on='number', how='left', suffixes=('', '_y')).set_index('time')
df['description'] = df['description_y']
df = df.drop('description_y', axis=1)
df.head()

In [ ]:
df_mapping_table

#### 指定した箇所のnumberでフィルタ、ピボットテーブルで各Number毎の1時間当たりの実績表にする

In [ ]:
# 特定の複数のnumberを指定します。
selected_numbers = {
    0:['0-0', '0-1', '0-2'],
    1:['1-1', '1-2', '1-3', '1-4', '1-5', '1-6', '1-7', '1-8'],
    2:['2-1', '2-2', '2-3', '2-4'],
    3:['3-1', '3-2', '3-3', '3-4'],
    4:['4-1', '4-2', '4-3'],
    5:['5-1', '5-2', '5-3', '5-4'],
    6:['6-1', '6-2', '6-3']}

# 特定の複数のnumberについてフィルタリングします。
df_filtered = df[df['number'].isin(selected_numbers[6])]

# time列をdatetime型に変換し、1時間ごとにグループ化してvalueの合計値を計算します。
df_result = df_filtered.groupby([pd.Grouper(level='time', freq='H'), 'number'])['value'].sum()
df_result = df_result.reset_index().pivot(index='time', columns='number', values='value')
df_result.head()

### 週次データ集計および折れ線グラフ化

In [ ]:
# time列をdatetime型に変換し、週単位にグループ化してvalueの合計値を計算します。
df_result_weekly = df_filtered.groupby([pd.Grouper(level='time', freq='W-MON'), 'number'])['value'].sum().reset_index(level=1)
df_result_weekly = df_result_weekly.pivot(columns='number', values='value')
# 指定したnumber毎に折れ線グラフを作成します。
df_result_weekly.plot(kind='line')

In [ ]:
def plot_weekly_sum(df, selected_numbers, value_col):
    """
    指定したnumber毎に折れ線グラフを週単位で作成する関数

    Parameters
    ----------
    df : DataFrame
        入力データフレーム
    selected_numbers : list of str
        指定するnumberのリスト
    value_col : str
        値を表す列名

    Returns
    -------
    None
    """
    
    # 特定の複数のnumberについてフィルタリングします。
    df_filtered = df[df['number'].isin(selected_numbers)]

    # time列をdatetime型に変換し、1時間ごとにグループ化してvalueの合計値を計算します。
    df_result = df_filtered.groupby([pd.Grouper(level='time', freq='H'), 'number'])[value_col].sum().reset_index(name=value_col)

    # time列をdatetime型に変換し、週単位にグループ化してvalueの合計値を計算します。
    df_result_weekly = df_result.groupby([pd.Grouper(key='time', freq='W'), 'number'])[value_col].sum().reset_index(name=value_col)

    # 指定したnumber毎に折れ線グラフを作成します。
    sns.lineplot(data=df_result_weekly, x='time', y=value_col, hue='number')

# 値を表す列名を指定します。
value_col = 'value'

# 関数を呼び出して折れ線グラフを作成します。
plot_weekly_sum(df, selected_numbers[6], value_col)
plt.show()

In [ ]:
def plot_monthly_sum(df, selected_numbers, value_col):
    """
    指定したnumber毎に折れ線グラフを月単位で作成する関数

    Parameters
    ----------
    df : DataFrame
        入力データフレーム
    selected_numbers : list of str
        指定するnumberのリスト
    value_col : str
        値を表す列名

    Returns
    -------
    None
    """
    
    # 特定の複数のnumberについてフィルタリングします。
    df_filtered = df[df['number'].isin(selected_numbers)]

    # time列をdatetime型に変換し、1時間ごとにグループ化してvalueの合計値を計算します。
    df_result = df_filtered.groupby([pd.Grouper(level='time', freq='H'), 'number'])[value_col].sum().reset_index(name=value_col)

    # time列をdatetime型に変換し、月単位にグループ化してvalueの合計値を計算します。
    df_result_monthly = df_result.groupby([pd.Grouper(key='time', freq='M'), 'number'])[value_col].sum().reset_index(name=value_col)

    # 指定したnumber毎に折れ線グラフを作成します。
    sns.lineplot(data=df_result_monthly, x='time', y=value_col, hue='number')
    
# 特定の複数のnumberを指定します。
selected_numbers = ['6-1', '6-2', '6-3']

# 値を表す列名を指定します。
value_col = 'value'

# 関数を呼び出して折れ線グラフを作成します。
plot_monthly_sum(df, selected_numbers, value_col)
plt.show()

#### selected_numbersで6を指定したときのcolumnの処理

In [ ]:
# outletを追加
df_result['outlet'] = df_result.apply(lambda row: row['6-1'] + row['6-3'], axis=1)
df_result['all'] = df_result.apply(lambda row: row['6-1'] + row['6-2'] + row['6-3'], axis=1)

mapping = ['3F', '2F', '1F', 'outlet', 'all']
df_result.columns = mapping
df_result.head()

#### 実績元データをpklファイルへ変換

In [ ]:
# df_result.to_pickle('/workspaces/data/flow/6.pkl')

#### 1.週間の1時間ごとの実績データグラフ作成用関数   2.各時間帯(1時間ごと)の各系列の実績データの分布グラフ作成関数

In [ ]:
def save_graphs_and_data(result: pd.DataFrame, save_path: str, start_date: str, end_date: str):
    # 日付範囲を指定
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # グラフサイズを指定
    plt.figure(figsize=(20,5))

    # グラフを作成
    current_date = start_date
    while current_date <= end_date:
        week_end_date = current_date + pd.Timedelta(days=6)
        week_data = result.loc[current_date:week_end_date]
        ax = week_data.plot(title=f'{current_date.date()} - {week_end_date.date()}', linewidth=1)

        # グラフを保存
        plt.savefig(f'{save_path}/{current_date.date()}-{week_end_date.date()}.png')

        current_date += pd.Timedelta(days=7)

    plt.show()

    # データフレームをエクセルファイルに保存
    result.to_excel(f'{save_path}/result.xlsx')
save_graphs_and_data(df_result, '/workspaces/data/flow/graph/', '2022-10-31', '2023-01-09')

def visualize_distribution(df: pd.DataFrame, save_path: str, start_date: str, end_date: str):
    # 日付範囲を指定
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    df = df.loc[start_date:end_date]
    df = df.reset_index()
    df['hour'] = df['time'].dt.hour
    
    cols = [col for col in df.columns if col not in ['time', 'hour']]
    n_cols = len(cols)
    
    for hour in range(24):
        hour_df = df[df['hour'] == hour]
        fig, axes = plt.subplots(n_cols, 1, figsize=(6, 4 * n_cols))
        
        for i in range(n_cols):
            col_name = cols[i]
            sns.swarmplot(x='hour', y=col_name, data=hour_df[hour_df[col_name] != 0], ax=axes[i])
        
        file_name = f'hour_{hour}.png'
        file_path = os.path.join(save_path, file_name)
        plt.savefig(file_path)
visualize_distribution(df_result,'/workspaces/data/flow/graph/hour/', '2022-10-31', '2023-01-09')

#### excelファイルをDataFrameへ変換する関数

In [ ]:
def convert_excel_to_df(year: int, month: int, path: str, startarea: str, endarea: str) -> pd.DataFrame:
    start_date = datetime(year, month, 1)
    end_date = (start_date + timedelta(days=31)).replace(day=1) - timedelta(days=1)
    date_list = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

    book = load_workbook(path, data_only=True)

    df = pd.DataFrame(index=pd.date_range(start=start_date.replace(hour=8), end=end_date.replace(hour=22), freq='H'),
                      columns=['ipa', 'sto', 'han', 'div', 'col', 'ent', 'alt'])
    df.index.name = 'time'

    for date in date_list:
        sheet_name = f'{date.day}日'
        if sheet_name not in book.sheetnames:
            continue

        sheet = book[sheet_name]

        data = [[cell.value for cell in row] for row in sheet[startarea:endarea]]

        col_name = ['ipa', 'sto', 'han', 'div', 'col', 'ent', 'alt']
        for i in range(len(col_name)):
            df.loc[(df.index.date == date.date()) & (df.index.hour >= 8) & (df.index.hour <= 17), col_name[i]] = data[i][:10]
            if data[i][10] is not None and data[i][12] is not None:
                df.loc[(df.index.date == date.date()) & (df.index.hour == 18), col_name[i]]=(data[i][10]+data[i][12])/2
            elif data[i][10] is not None:
                df.loc[(df.index.date == date.date()) & (df.index.hour == 18), col_name[i]]=data[i][10]/2
            elif data[i][12] is not None:
                df.loc[(df.index.date == date.date()) & (df.index.hour == 18), col_name[i]]=data[i][12]/2    
            df.loc[(df.index.date == date.date()) & (df.index.hour >= 19) & (df.index.hour <= 21), col_name[i]] = data[i][13:16]
            df.loc[(df.index.date == date.date()) & (df.index.hour == 22), col_name[i]] = data[i][17]
            df.loc[(df.index.date == date.date()) & (df.index.hour == 23), col_name[i]] = data[i][18]
            df.loc[(df.index.date == date.date() + timedelta(days=1)) & (df.index.hour >= 0) & (df.index.hour <= 6), col_name[i]] = data[i][19:26]

    return df.fillna(0)

year = 2023
month =3
path = '/workspaces/data/flow/【QPS】作業計画(3月).xlsx'
startarea = 'E27'
endarea = 'AD33'

df1 = convert_excel_to_df(year, month, path, startarea, endarea)
df = pd.read_pickle('/workspaces/data/flow/mhNovDecJanFebMar.pkl')
df = df.loc[(df.index < '2023-03-1 00:00:00')]
df = pd.concat([df, df1], axis=0)

In [ ]:
df.loc[df.index < '2023-03-24 00:00:00'].tail(24)

In [ ]:
# df.to_pickle('/workspaces/data/flow/mhNovDecJanFebMar.pkl')

#### pklファイルから実績データ読み出し結合

In [ ]:
df_flow = pd.read_pickle('/workspaces/data/flow/6.pkl')
df_flow.head()
df_mh = pd.read_pickle('/workspaces/data/flow/mhNovDecJanFebMar.pkl')
df_mh.head()
# pklファイルからDataFrame読み出して1つのDataFrameにマージ
df = df_flow.merge(df_mh, how='outer', on='time').fillna(0)
# asfreqメソッドを使用して欠損値をNaNで埋めます。
df = df.resample('1H').asfreq()
# locメソッドによりdfの期間指定
df = df.loc[(df.index >= '2022-11-1 07:00:00') & (df.index < '2023-03-23 23:00:00')]
# 最後にfillnaメソッドを使用してNaNを0で埋めます。
df = df.fillna(0)

In [ ]:
df['all'].sum()/df[['ipa', 'sto', 'han', 'div', 'col', 'ent', 'alt']].sum().sum()

In [ ]:
df['3F_efi'] = df.apply(lambda row: row['3F']/ (row['sto'] + row['han'] + row['alt']) if (row['sto'] + row['han'] + row['alt']) != 0 else 0 , axis=1)
df['2F_efi'] = df.apply(lambda row: row['2F']/ (row['div'] + row['col']) if (row['div'] + row['col']) != 0 else 0, axis=1)
df['1F_efi'] = df.apply(lambda row: row['1F']/ row['ipa'] if row['ipa'] != 0 else 0, axis=1)

df['all_efi'] = df.apply(lambda row: row['all']/ (row['sto'] + row['han'] + row['alt'] + row['div'] + row['col'] + row['ipa']) if (row['sto'] + row['han'] + row['alt'] + row['div'] + row['col'] + row['ipa']) != 0 else 0 , axis=1)

#### IQR（四分位範囲）をもとに外れ値除去(使用しない)

In [ ]:
def remove_spikes(df: pd.DataFrame, select_col: list) -> pd.DataFrame:
    for col in select_col:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

selecte_col = ['3F_efi', '2F_efi', '1F_efi']

df = remove_spikes(df, select_col=selecte_col)

階差をもとにスパイク除去   
指定した列のデータ列からスパイク部分を検知  
スパイクがある行は指定列に対応する列(複数可)を0で埋める

In [ ]:
def replace_spikes_dict(df: pd.DataFrame, select_col: dict, threshold: float) -> pd.DataFrame:
    df = df.copy()
    for col, related_cols in select_col.items():
        mask = df[col].diff().abs() > threshold
        mask |= df[col].diff(+1).abs() > threshold
        index = np.where(mask)[0]
        for i in index:
            if i == 0 or i == len(df) - 1:
                continue
            # df.iloc[i][col] = (df.iloc[i-1][col] + df.iloc[i+1][col]) / 2
            for related_col in related_cols:
                # df.iloc[i][related_col] = (df.iloc[i-1][related_col] + df.iloc[i+1][related_col]) / 2
                df.iloc[i][related_col] = 0
    return df
# 入力されたdfのselect_colのkeyで指定された列でスパイク値がある行について、keyに対応する列（複数可）をスパイクの行は0で埋める

selecte_col = {
    '3F_efi':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '2F_efi':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '1F_efi':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    'all_efi':['ipa', 'sto', 'han', 'div', 'col', 'alt']}

# 手入力でのデータがご入力が多いため、個別0埋め
df = replace_spikes_dict(df, selecte_col, 350)
df['3F_efi'] = df.apply(lambda row: row['3F']/ (row['sto'] + row['han'] + row['alt']) if (row['sto'] + row['han'] + row['alt']) != 0 else 0 , axis=1)
df['2F_efi'] = df.apply(lambda row: row['2F']/ (row['div'] + row['col']) if (row['div'] + row['col']) != 0 else 0, axis=1)
df['1F_efi'] = df.apply(lambda row: row['1F']/ row['ipa'] if row['ipa'] != 0 else 0, axis=1)
df['all_efi'] = df.apply(lambda row: row['all_efi'] if (row['sto'] + row['han'] + row['alt'] + row['div'] + row['col'] + row['ipa']) != 0 else 0 , axis=1)
df['3F'] = df.apply(lambda row: 0 if (row['sto'] + row['han'] + row['alt']) == 0 else row['3F'] , axis=1)
df['2F'] = df.apply(lambda row: 0 if (row['div'] + row['col']) == 0  else row['2F'] , axis=1)
df['1F'] = df.apply(lambda row: 0 if row['ipa'] == 0 else row['1F'], axis=1)
df['all'] = df.apply(lambda row: 0 if (row['sto'] + row['han'] + row['alt'] + row['div'] + row['col'] + row['ipa']) == 0 else row['all'] , axis=1)
df['outlet'] = df.apply(lambda row: 0 if (row['sto'] + row['han'] + row['alt'] + row['div'] + row['col'] + row['ipa']) == 0 else row['all'] , axis=1)

In [ ]:
df = df[(df.T == 0).any()]
df[df.index >= '2022-11-05 0:00:00'].head(60)

In [ ]:
df_t = df.loc[(df.index >= '2023-02-01 00:00:00') & (df.index <= '2023-04-1 00:00:00')]
plt.figure(figsize=(50, 5))
df_t = df_t.reset_index().melt('time', var_name='cols', value_name='vals')
df_t = df_t[df_t['cols'].isin(['3F_efi', '2F_efi', '1F_efi', 'all_efi'])]
sns.lineplot(x='time', y='vals', hue='cols', data=df_t, linewidth=1)

# 横軸の設定
ax = plt.gca()
hours = mdates.HourLocator(interval=4)
h_fmt = mdates.DateFormatter('%H時')
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(h_fmt)

# 横軸ラベルの文字サイズ設定
ax.tick_params(axis='x', labelsize=6)

# plt.show()
plt.savefig('/workspaces/data/flow/plot.jpg')

## 前処理済みデータpklファイルロード

In [ ]:
# df.to_pickle('/workspaces/data/flow/preprocessed_dataset.pkl')
df = pd.read_pickle('/workspaces/data/flow/preprocessed_dataset.pkl')

In [ ]:
num = 2
print(len(df[(df['alt'] < num)].apply(lambda col:col[col != 0])['1F_efi']))
df_t = df[(df['alt'] < num)].apply(lambda col:col[col != 0])['1F_efi'].dropna()
print(len(df[((df['alt'] > num))].apply(lambda col:col[col != 0])['1F_efi']))
df_tt = df[((df['alt'] > num))].apply(lambda col:col[col != 0])['1F_efi'].dropna()

print(df_t.describe())
print(df_tt.describe())

# ウェルチのt検定
t_stat, p_value = stats.ttest_ind(df_t, df_tt, equal_var=False)
print("Welch's t-test: p-value =", p_value)

# マン・ホイットニーのU検定
u_stat, p_value = stats.mannwhitneyu(df_t, df_tt, alternative='two-sided')
print("Mann-Whitney U test: p-value =", p_value)
# ヒストグラムとKDEプロットを重ねて描画
sns.histplot(df_t, kde=True, color='blue', alpha=0.5, label='< num')
sns.histplot(df_tt, kde=True, color='red', alpha=0.5, label='> num')

# グラフのタイトルと凡例を追加
plt.title('Distribution of Group 1 and Group 2')
plt.legend()

#### ロバスト回帰分析(フーバー回帰)を実行し、各種プロットする。

In [ ]:
def run_robust_regression_and_residual_analysis(data, var_dict, save_path, epsilon=1.35):
    # 入力検証
    validate_input(var_dict, data)

    # 指定されたパスにフォルダがなければ作成
    create_folder(save_path)

    output = {}
    for y_var, x_vars in var_dict.items():
        # 説明変数と目的変数の設定
        X = data[x_vars]
        y = data[y_var]

        # 回帰分析の実行
        model = HuberRegressor(epsilon=epsilon)
        model.fit(X,y)
        
        output[y_var] = model
        
        print(f"Results for {y_var}:\n")
        print('Coefficients: \n', model.coef_)

        coefs_df = pd.DataFrame({'variable': x_vars,'coefficient': model.coef_})
        coefs_df.to_csv(os.path.join(save_path,f'{y_var}_coefficients_robustlinear_{epsilon}.csv'),index=False)
        
        # 残差プロット
        residuals = y - model.predict(X)
        
        # 標準化残差プロット
        standardized_residuals = (residuals - residuals.mean()) / residuals.std()
        plt.scatter(model.predict(X), standardized_residuals)
        plt.xlabel('Fitted values')
        plt.ylabel('Standardized residuals')
        plt.savefig(os.path.join(save_path,f'{y_var}_standardized_residuals_robustlinear_{epsilon}.png'))
        plt.show()
        
        # Q-Qプロット
        stats.probplot(standardized_residuals, dist="norm", plot=plt)
        plt.savefig(os.path.join(save_path,f'{y_var}_qqplot_robustlinear_{epsilon}.png'))
        plt.show()
        
        # 残差 vs. 説明変数プロット 
        fig, ax = plt.subplots(1,len(x_vars), figsize=(30, 5))
        for i in range(len(x_vars)):
            if len(x_vars) > 1:
                ax[i].scatter(data[x_vars[i]], residuals)
                ax[i].set_xlabel(x_vars[i])
                ax[i].set_ylabel('Residuals')
            else:
                ax.scatter(data[x_vars[i]], residuals)
                ax.set_xlabel(x_vars[i])
                ax.set_ylabel('Residuals')
        plt.savefig(os.path.join(save_path,f'{y_var}_residuals_vs_explanatory_robustlinear_{epsilon}.png'))
        plt.show()
    return output

def validate_input(var_dict,data):
    if not isinstance(var_dict, dict):
        raise ValueError("var_dict must be a dictionary")
    for y_var,x_vars in var_dict.items():
        if not isinstance(y_var,str) or not isinstance(x_vars,list):
            raise ValueError("var_dict keys must be strings and values must be lists")
        if y_var not in data.columns:
            raise ValueError(f"{y_var} is not a column in the data")
        for x_var in x_vars:
            if x_var not in data.columns:
                raise ValueError(f"{x_var} is not a column in the data")

def create_folder(save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

# データの読み込み
data = df.loc[(df.index >= '2023-02-01 00:00:00') & (df.index <= '2023-03-1 00:00:00')]

var_dict = {
    '3F':['sto', 'han', 'alt'],
    '2F':['div', 'col'],
    '1F':['ipa'],
    'all':['ipa', 'sto', 'han', 'div', 'col', 'alt']}

save_path = "/workspaces/data/flow/robustlinearFeb_scikit"

run_robust_regression_and_residual_analysis(data, var_dict, save_path, 2.0)

In [ ]:
def run_sm_regression_and_residual_analysis(data, var_dict, save_path):
    # 入力検証
    validate_input(var_dict,data)

    output = {}

    # 指定されたパスにフォルダがなければ作成
    create_folder(save_path)

    for y_var, x_vars in var_dict.items():
        # 説明変数と目的変数の設定
        X = data[x_vars]
        y = data[y_var]

        # 回帰分析の実行
        X = sm.add_constant(X)
        model = sm.OLS(y,X)
        output[y_var] = model
        results = model.fit()

        print(f"Results for {y_var}:\n")
        print(results.summary())

        # 回帰係数の優位性の確認
        print(f"\nP-values for {y_var}:\n")
        print(results.pvalues)

        # 回帰係数をcsvファイルとして保存
        save_coefficients(results,y_var,save_path)

        # 標準化残差プロット
        plot_standardized_residuals(results,y_var,save_path)

        # Q-Qプロット
        plot_qqplot(results,y_var,save_path)

        # 残差 vs. 説明変数プロット
        plot_residuals_vs_explanatory(data,x_vars,y_var,results,save_path,output)
            
    return output

def validate_input(var_dict,data):
    if not isinstance(var_dict, dict):
        raise ValueError("var_dict must be a dictionary")
    for y_var,x_vars in var_dict.items():
        if not isinstance(y_var,str) or not isinstance(x_vars,list):
            raise ValueError("var_dict keys must be strings and values must be lists")
        if y_var not in data.columns:
            raise ValueError(f"{y_var} is not a column in the data")
        for x_var in x_vars:
            if x_var not in data.columns:
                raise ValueError(f"{x_var} is not a column in the data")

def create_folder(save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

def save_coefficients(results,y_var,save_path):
    coefs_df = pd.DataFrame({'variable':results.params.index,'coefficient':results.params.values})
    coefs_df.to_csv(os.path.join(save_path,f'{y_var}_coefficients.csv'),index=False)

def plot_standardized_residuals(results,y_var,save_path):
    plt.scatter(results.fittedvalues, results.get_influence().resid_studentized_internal)
    plt.xlabel('Fitted values')
    plt.ylabel('Standardized residuals')
    plt.savefig(os.path.join(save_path,f'{y_var}_standardized_residuals.png'))
    plt.show()

def plot_qqplot(results,y_var,save_path):
    sm.qqplot(results.get_influence().resid_studentized_internal,line='45')
    plt.savefig(os.path.join(save_path,f'{y_var}_qqplot.png'))
    plt.show()

def plot_residuals_vs_explanatory(data,x_vars,y_var,results,save_path,output):
    fig, ax = plt.subplots(1,len(x_vars), figsize=(20, 5))
    for i in range(len(x_vars)):
        if len(x_vars) > 1:
            ax[i].scatter(data[x_vars[i]], results.resid)
            ax[i].set_xlabel(x_vars[i])
            ax[i].set_ylabel('Residuals')
        else:
            ax.scatter(data[x_vars[i]], results.resid)
            ax.set_xlabel(x_vars[i])
            ax.set_ylabel('Residuals')
    plt.savefig(os.path.join(save_path,f'{y_var}_residuals_vs_explanatory.png'))
    plt.show()

# データの読み込み
data = df.loc[(df.index >= '2023-02-01 00:00:00') & (df.index <= '2023-03-1 00:00:00')]

var_dict = {
    '3F':['sto', 'han', 'alt'],
    '2F':['div', 'col'],
    '1F':['ipa'],
    'all':['ipa', 'sto', 'han', 'div', 'col', 'alt']}

save_path = "/workspaces/data/flow/linearFeb_sm"

# 重回帰分析と残差診断の実行
models = run_sm_regression_and_residual_analysis(data, var_dict, save_path)

#### 線形重回帰分析(scikit_learn)を実行し、各種プロットする

In [ ]:
def run_scikit_regression_and_residual_analysis(data, var_dict, save_path):
    # 入力検証
    validate_input(var_dict,data)

    output = {}

    # 指定されたパスにフォルダがなければ作成
    create_folder(save_path)

    for y_var, x_vars in var_dict.items():
        # 説明変数と目的変数の設定
        X = data[x_vars]
        y = data[y_var]

        # 回帰分析の実行
        model = LinearRegression()
        model.fit(X,y)
        
        output[y_var] = model

        # 結果の表示
        print(f"Results for {y_var}:\n")
        print('Coefficients: \n', model.coef_)
        
        coefs_df = pd.DataFrame({'variable': x_vars,'coefficient': model.coef_})
        coefs_df.to_csv(os.path.join(save_path,f'{y_var}_coefficients_linear.csv'),index=False)

        # 残差プロット
        residuals = y - model.predict(X)
        
        # 標準化残差プロット
        standardized_residuals = (residuals - residuals.mean()) / residuals.std()
        plt.scatter(model.predict(X), standardized_residuals)
        plt.xlabel('Fitted values')
        plt.ylabel('Standardized residuals')
        plt.savefig(os.path.join(save_path,f'{y_var}_standardized_residuals_linear.png'))
        plt.show()
        
        # Q-Qプロット
        stats.probplot(standardized_residuals, dist="norm", plot=plt)
        plt.savefig(os.path.join(save_path,f'{y_var}_qqplot_linear.png'))
        plt.show()
        
        # 残差 vs. 説明変数プロット
        fig, ax = plt.subplots(1,len(x_vars), figsize=(20, 5))
        for i in range(len(x_vars)):
            if len(x_vars) > 1:
                ax[i].scatter(data[x_vars[i]], residuals)
                ax[i].set_xlabel(x_vars[i])
                ax[i].set_ylabel('Residuals')
            else:
                ax.scatter(data[x_vars[i]], residuals)
                ax.set_xlabel(x_vars[i])
                ax.set_ylabel('Residuals')
        plt.savefig(os.path.join(save_path,f'{y_var}_residuals_vs_explanatory_linear.png'))
        plt.show()

    return output

def validate_input(var_dict,data):
    if not isinstance(var_dict, dict):
        raise ValueError("var_dict must be a dictionary")
    for y_var,x_vars in var_dict.items():
        if not isinstance(y_var,str) or not isinstance(x_vars,list):
            raise ValueError("var_dict keys must be strings and values must be lists")
        if y_var not in data.columns:
            raise ValueError(f"{y_var} is not a column in the data")
        for x_var in x_vars:
            if x_var not in data.columns:
                raise ValueError(f"{x_var} is not a column in the data")

def create_folder(save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

# データの読み込み
data = df.loc[(df.index >= '2023-02-01 00:00:00') & (df.index <= '2023-03-1 00:00:00')]

"""var_dict = {
    '3F':['sto', 'han', 'alt'],
    '2F':['div', 'col'],
    '1F':['ipa'],
    'all':['ipa', 'sto', 'han', 'div', 'col', 'alt']}"""

var_dict = {
    '3F':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '2F':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '1F':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    'all':['ipa', 'sto', 'han', 'div', 'col', 'alt']}

save_path = "/workspaces/data/flow/linearFeb_scikit"

# 重回帰分析と残差診断の実行
models = run_scikit_regression_and_residual_analysis(data, var_dict, save_path)

In [ ]:
def validate_regression_model(model_output, validation_data):
    """
    model_output: 辞書。キーは従属変数名であり、値はstatsmodels OLSモデルオブジェクトです。
    validation_data: 検証用のpandas DataFrame。
    """
    for y_var, model in model_output.items():
        X = validation_data[model.exog_names[1:]]
        X = sm.add_constant(X)
        y_pred = model.predict(exog=X, params=model.fit().params)
        print(f"Predictions for {y_var}:\n")
        print(y_pred)

# データの読み込み
data = df.loc[(df.index >= '2023-01-01 00:00:00') & (df.index <= '2023-02-1 00:00:00')]

var_dict = {
    '3F':['sto', 'han', 'alt'],
    '2F':['div', 'col'],
    '1F':['ipa'],
    'all':['ipa', 'sto', 'han', 'div', 'col', 'alt']}

validate_regression_model(models, data)

#### ランダムフォレストによる回帰分析を実行し、各種プロットしたグラフ画像を指定フォルダに保存する

In [ ]:
def run_tree_regression_and_residual_analysis(train_df, test_df, var_dict, param_grid, save_path):
    """
    説明変数と目的変数を指定して、複数の決定木回帰モデルを構築し、予測値を計算する関数。
    :param df: データフレーム。説明変数と目的変数が含まれる。
    :type df: pandas.DataFrame
    :param var_dict: 説明変数と目的変数の辞書。キーが目的変数名で、値が説明変数名のリスト。
    :type var_dict: dict of str to list of str
    :param save_path: グラフを保存するフォルダのパス。
    :type save_path: str
    :return: 構築したモデルと予測値の辞書
    :rtype: tuple of dict
    """
    # 入力検証
    validate_input(var_dict,train_df)
    validate_input(var_dict,test_df)

    # 指定されたパスにフォルダがなければ作成
    create_folder(save_path)

    models = {}
    train_y_preds = {}
    test_y_preds = {}

    # 結果を格納するDataFrame作成
    result_df = pd.DataFrame(columns=['y_var', 'train_mse', 'train_mae', 'train_r2', 'test_mse', 'test_mae', 'test_r2'])

    # 指定されたフォルダが存在しない場合は作成する。
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for y_var, x_vars in var_dict.items():
        # モデルインスタンス作成
        model = RandomForestRegressor()

        # グリッドサーチ実行
        grid_search = GridSearchCV(estimator=model,
                                   param_grid=param_grid,
                                   cv=3,
                                   n_jobs=-1)
        grid_search.fit(train_df[x_vars], train_df[y_var])

        # 最適なパラメータでモデル再学習
        best_model = grid_search.best_estimator_
        models[y_var] = best_model


        # 学習用データでの予測と評価指標計算
        train_y_pred = best_model.predict(train_df[x_vars])
        train_y_preds[y_var] = train_y_pred
        train_mse = mean_squared_error(train_df[y_var], train_y_pred)
        train_mae = mean_absolute_error(train_df[y_var], train_y_pred)
        train_r2 = r2_score(train_df[y_var], train_y_pred)
        
        # 検証用データでの予測と評価指標計算
        test_y_pred = best_model.predict(test_df[x_vars])
        test_y_preds[y_var] = test_y_pred
        test_mse = mean_squared_error(test_df[y_var], test_y_pred)
        test_mae = mean_absolute_error(test_df[y_var], test_y_pred)
        test_r2 = r2_score(test_df[y_var], test_y_pred)

        # 結果のサマリ表示
        """
        print(f'{y_var} の学習用MSE:', train_mse)
        print(f'{y_var} の学習用MAE:', train_mae)
        print(f'{y_var} の学習用決定係数:', train_r2)

        print(f'{y_var} の検証用MSE:', test_mse)
        print(f'{y_var} の検証用MAE:', test_mae)
        print(f'{y_var} の検証用決定係数:', test_r2)"""
        
        # 結果をDataFrameに追加
        result_df.loc[len(result_df)] = [y_var, train_mse, train_mae, train_r2, test_mse, test_mae, test_r2]

        # 結果をCSVファイルに保存
        result_file_path = os.path.join(save_path, 'result.csv')
        result_df.to_csv(result_file_path,index=False)

        # 残差計算
        residuals = train_df[y_var] - train_y_pred

        # 標準化残差計算
        standardized_residuals = (residuals - residuals.mean()) / residuals.std()

        # グラフのプロット部
        """
        # 残差プロット表示および保存
        plt.scatter(train_y_pred, residuals)
        plt.xlabel('予測値')
        plt.ylabel('残差')
        plt.title(f'{y_var} の予測値 vs 残差')
        plt.savefig(os.path.join(save_path, f'{y_var}_residual_plot_tree_maxdepth{max_depth}.png'))
        plt.show()

        # 標準化残差プロット表示および保存
        plt.scatter(train_y_pred, standardized_residuals)
        plt.xlabel('予測値')
        plt.ylabel('標準化残差')
        plt.title(f'{y_var} の予測値 vs 標準化残差')
        plt.savefig(os.path.join(save_path, f'{y_var}_standardized_residual_plot_tree_maxdepth{max_depth}.png'))
        plt.show()

        # Q-Qプロット表示および保存
        stats.probplot(residuals, dist="norm", plot=plt)
        plt.title(f'{y_var} のQ-Qプロット')
        plt.savefig(os.path.join(save_path, f'{y_var}_qq_plot_tree_maxdepth{max_depth}.png'))
        plt.show()

        # 残差 vs. 説明変数プロット
        fig, ax = plt.subplots(1,len(x_vars), figsize=(20, 5))
        for i in range(len(x_vars)):
            if len(x_vars) > 1:
                ax[i].scatter(df[x_vars[i]], residuals)
                ax[i].set_xlabel(x_vars[i])
                ax[i].set_ylabel('Residuals')
            else:
                ax.scatter(df[x_vars[i]], residuals)
                ax.set_xlabel(x_vars[i])
                ax.set_ylabel('Residuals')
        plt.savefig(os.path.join(save_path,f'{y_var}_residuals_vs_explanatory_tree_maxdepth{max_depth}.png'))
        plt.show()
        """

    return models

def validate_input(var_dict,data):
    if not isinstance(var_dict, dict):
        raise ValueError("var_dict must be a dictionary")
    for y_var,x_vars in var_dict.items():
        if not isinstance(y_var,str) or not isinstance(x_vars,list):
            raise ValueError("var_dict keys must be strings and values must be lists")
        if y_var not in data.columns:
            raise ValueError(f"{y_var} is not a column in the data")
        for x_var in x_vars:
            if x_var not in data.columns:
                raise ValueError(f"{x_var} is not a column in the data")

def create_folder(save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

# データの読み込み
train_df = df.loc[(df.index >= '2022-12-01 00:00:00') & (df.index < '2023-02-1 00:00:00')]
test_df = df.loc[(df.index >= '2023-02-01 00:00:00') & (df.index < '2023-03-1 00:00:00')]

var_dict = {
    '3F':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '2F':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '1F':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    'all':['ipa', 'sto', 'han', 'div', 'col', 'alt']}

# グリッドサーチ用パラメータ設定
param_grid = {
    'n_estimators': [30, 50, 80],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [10, 20, 30],
    'min_samples_split': [0.02]
}

save_path = "/workspaces/data/flow/RFtrain1201test0233"

models = run_tree_regression_and_residual_analysis(train_df, test_df, var_dict, param_grid, save_path)

In [ ]:
def cross_validate(df, var_dict, save_path):
    kf = KFold(n_splits=5)
    models = {}
    for train_index, test_index in kf.split(df):
        train_df = df.iloc[train_index]
        test_df = df.iloc[test_index]
        models.update(run_tree_regression_and_residual_analysis(train_df, test_df, var_dict, save_path))
    return models

In [ ]:
# vote averageが下位50%だったらlow, 上位50%だったらhigh
sns.pairplot(df[['ipa', 'sto', 'han', 'div', 'col', 'alt']],
             plot_kws={'alpha':0.3},
             #diag_kind='hist'
            )

#### 勾配ブースティング(XGBRegressor)のハイパーパラメータのあたり付け

In [ ]:
# データの読み込み
train_df = df.loc[(df.index >= '2022-12-01 00:00:00') & (df.index < '2023-02-1 00:00:00')].reset_index()
test_df = df.loc[(df.index >= '2023-02-01 00:00:00') & (df.index < '2023-03-1 00:00:00')].reset_index()

USE_EXPLANATORY = ['ipa', 'sto', 'han', 'alt']
OBJECTIVE_VARIALBLE = '3F'

X = train_df[USE_EXPLANATORY].values
Y = train_df[OBJECTIVE_VARIALBLE]

#  乱数シード
seed = 23
# モデル作成
model = XGBRegressor(booster='gbtree', objective='reg:absoluteerror', random_state=seed, n_estimators=10000)
# 学習時fitパラメータ指定
fit_params = {'verbose': False,
                'early_stopping_rounds': 15,
                'eval_metric': 'mae',
                'eval_set': [([X, Y])]}

cv = KFold(n_splits=3, shuffle=True, random_state=seed)
"""
regplot.regression_heat_plot(model, USE_EXPLANATORY, OBJECTIVE_VARIALBLE, train_df,
                             pair_sigmarange = 0.5, rounddigit_x1=3, rounddigit_x2=3,
                             cv=cv, display_cv_indices=0,
                             fit_params=fit_params)
"""

scoring = 'neg_mean_absolute_error'
scores = cross_val_score(model, X, Y, cv=cv, scoring=scoring, n_jobs=-1, fit_params=fit_params)
print(f'scores={scores}')
print(f'average_score={np.mean(scores)}')

cv_params = {'subsample': [0, 0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9, 1.0],
             'colsample_bytree': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             'reg_alpha': [0, 0.0001, 0.001, 0.01, 0.03, 0.1, 0.3, 1.0],
             'reg_lambda': [0, 0.0001, 0.001, 0.01, 0.03, 0.1, 0.3, 1.0],
             'learning_rate': [0, 0.0001, 0.001, 0.01, 0.03, 0.1, 0.3, 1.0],
             'min_child_weight': [1, 3, 5, 7, 9, 11, 13, 15],
             'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
             'gamma': [0, 0.0001, 0.001, 0.01, 0.03, 0.1, 0.3, 1.0]
             }
param_scales = {'subsample': 'linear',
                'colsample_bytree': 'linear',
                'reg_alpha': 'log',
                'reg_lambda': 'log',
                'learning_rate': 'log',
                'min_child_weight': 'linear',
                'max_depth': 'linear',
                'gamma': 'log'
                }

# 検証曲線のプロット（パラメータ毎にプロット）
for i, (k, v) in enumerate(cv_params.items()):
    train_scores, valid_scores = validation_curve(estimator=model,
                                                  X=X, y=Y,
                                                  param_name=k,
                                                  param_range=v,
                                                  fit_params=fit_params,
                                                  cv=cv, scoring=scoring,
                                                  n_jobs=-1)
    # 学習データに対するスコアの平均±標準偏差を算出
    train_mean = np.mean(train_scores, axis=1)
    train_std  = np.std(train_scores, axis=1)
    train_center = train_mean
    train_high = train_mean + train_std
    train_low = train_mean - train_std
    # テストデータに対するスコアの平均±標準偏差を算出
    valid_mean = np.mean(valid_scores, axis=1)
    valid_std  = np.std(valid_scores, axis=1)
    valid_center = valid_mean
    valid_high = valid_mean + valid_std
    valid_low = valid_mean - valid_std
    # training_scoresをプロット
    plt.plot(v, train_center, color='blue', marker='o', markersize=5, label='training score')
    plt.fill_between(v, train_high, train_low, alpha=0.15, color='blue')
    # validation_scoresをプロット
    plt.plot(v, valid_center, color='green', linestyle='--', marker='o', markersize=5, label='validation score')
    plt.fill_between(v, valid_high, valid_low, alpha=0.15, color='green')
    # スケールをparam_scalesに合わせて変更
    plt.xscale(param_scales[k])
    # 軸ラベルおよび凡例の指定
    plt.xlabel(k)  # パラメータ名を横軸ラベルに
    plt.ylabel(scoring)  # スコア名を縦軸ラベルに
    plt.legend(loc='lower right')  # 凡例
    # グラフを描画
    plt.show()

勾配ブースティングのための関数群

In [ ]:
def validate_input(target_to_features, data):
    """
    入力変数とデータ構造を検証します。
    Args:
        target_to_features (dict): 目的変数と対応する説明変数のディクショナリ。
        data (pd.DataFrame): 入力データ。
    Raises:
        ValueError: 入力が無効な場合。
    """
    if not isinstance(target_to_features, dict):
        raise ValueError("target_to_features must be a dictionary")
    for target, features in target_to_features.items():
        if not isinstance(target, str) or not isinstance(features, list):
            raise ValueError("target_to_features keys must be strings and values must be lists")
        if target not in data.columns:
            raise ValueError(f"{target} is not a column in the data")
        for feature in features:
            if feature not in data.columns:
                raise ValueError(f"{feature} is not a column in the data")

def create_directory_if_not_exists(directory_path):
    """
    指定されたパスにフォルダが存在しない場合、フォルダを作成します。
    Args:
        directory_path (str): フォルダを作成するパス。
    """
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

def add_hour_column(df):
    """
    'time'列から時刻を抽出し、新しい列としてDataFrameに追加します。

    Args:
        df (pd.DataFrame): 入力データ。

    Returns:
        pd.DataFrame: 新しい 'hour' 列が追加されたDataFrame。
    """
    df['hour'] = pd.to_datetime(df['time']).dt.hour
    return df

def one_hot_encode_hours(df):
    """
    'hour' 列をワンホットエンコードし、変更されたDataFrameを返します。
    Args:
        df (pd.DataFrame): 入力データ。
    Returns:
        pd.DataFrame: ワンホットエンコードされた 'hour' 列を持つDataFrame。
    """
    df = pd.get_dummies(df, columns=['hour'], prefix='hour', drop_first=True)
    return df

def load_hyperparameters_from_csv(params_directory, target):
    """
    与えられた目的変数のCSVファイルからハイパーパラメータを読み込みます。
    Args:
        params_directory (str): ハイパーパラメータを含むCSVファイルがあるフォルダへのパス。
        target (str): 目的変数名。
    Returns:
        dict: 目的変数のハイパーパラメータを含むディクショナリ。
    """
    file_path = os.path.join(params_directory, f'best_params_{target}.csv')
    df = pd.read_csv(file_path, index_col=0)

    # Convert values to appropriate types
    hyperparams = {}
    for index, row in df.iterrows():
        if index in ['max_depth', 'min_child_weight']:
            hyperparams[index] = int(row['value'])
        elif index in ['colsample_bytree', 'learning_ration', 'subsample']:
            hyperparams[index] = float(row['value'])
    return hyperparams

def plot_actual_vs_predicted_and_residuals(train_df, test_df, target, features, train_y_pred, test_y_pred, residuals, save_directory, eval_metric):
    """
    実際の値と予測値のグラフおよび残差グラフをプロットし、保存する関数。
    Args:
        train_df (pd.DataFrame): 学習用データセット。
        test_df (pd.DataFrame): 検証用データセット。
        target (str): 目的変数。
        features (list): 説明変数のリスト。
        train_y_pred (np.array): 学習用データの予測値。
        test_y_pred (np.array): 検証用データの予測値。
        residuals (np.array): 学習用データの残差。
        save_directory (str): 結果を保存するディレクトリへのパス。
        eval_metric (str): 評価指標。
    """
    # 観測値、予測値 vs. time
    fig, axs = plt.subplots(2, 1, figsize=(50, 16))
    axs[0].plot(train_df['time'], train_df[target], label='Actual')
    axs[0].plot(train_df['time'], train_y_pred, label='Predicted')
    axs[0].set_title('Train Data')
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel(target)
    axs[0].legend()
    
    axs[1].plot(test_df['time'], test_df[target], label='Actual')
    axs[1].plot(test_df['time'], test_y_pred, label='Predicted')
    axs[1].set_title('Test Data')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel(target)
    axs[1].legend()
    
    fig.suptitle(f'{target} - Actual vs Predicted')
    plt.savefig(os.path.join(save_directory, f'{target}_yy_vs_time_{eval_metric}.png'))
    plt.close(fig)

    # 残差 vs. 説明変数
    fig, ax = plt.subplots(1, len(features), figsize=(10*len(features), 20))
    for i in range(len(features)):
        if len(features) > 1:
            ax[i].scatter(train_df[features[i]], residuals)
            ax[i].set_xlabel(features[i])
            ax[i].set_ylabel('Residuals')
        else:
            ax.scatter(train_df[features[i]], residuals)
            ax.set_xlabel(features[i])
            ax.set_ylabel('Residuals')
    plt.savefig(os.path.join(save_directory,f'{target}_residuals_vs_xvars_{eval_metric}.png'))
    plt.close(fig)

def predict_and_evaluate(models, df, target_to_features, save_path):
    """
    与えられたモデルを使って、指定されたDataFrameに対して予測を行い、各種評価指標とグラフを表示します。
    Args:
        models (dict): ターゲット変数ごとに訓練されたモデルの辞書
        df (pd.DataFrame): 予測を行う対象のデータフレーム
        target_to_features (dict): ターゲット変数ごとに使用する説明変数のリストを格納した辞書, 説明変数にはOne Hot Encodingされて生成された時間列含む。
        save_path (str): グラフを保存するディレクトリのパス
    """
    # 時間（Hour）列を作成
    df = add_hour_column(df)
    
    # 時間列に対してOne Hot Encodingを実行
    df = one_hot_encode_hours(df)

    # 結果を格納するDataFrameを作成
    result_df = pd.DataFrame(columns=['y_var', 'mse', 'mae', 'r2'])

    for target, features in target_to_features.items():
        # 予測値を計算
        y_pred = models[target].predict(df[features])

        # 評価指標を計算
        mse = mean_squared_error(df[target], y_pred)
        mae = mean_absolute_error(df[target], y_pred)
        r2 = r2_score(df[target], y_pred)

        # 結果をDataFrameに追加
        result_df.loc[len(result_df)] = [target, mse, mae, r2]

        # グラフの作成
        fig, ax = plt.subplots(figsize=(15, 6))
        ax.plot(df['time'], df[target], label='Actual')
        ax.plot(df['time'], y_pred, label='Predicted')
        ax.set_title(f'{target} - Actual vs Predicted')
        ax.set_xlabel('Time')
        ax.set_ylabel(target)
        ax.legend()

        # グラフの保存
        plt.savefig(os.path.join(save_path, f'{target}_prediction.png'))
        plt.close(fig)

    # 結果の表示
    print(result_df)

    # 結果の保存
    result_file_path = os.path.join(save_path, 'prediction_result.csv')
    result_df.to_csv(result_file_path, index=False)


#### 勾配ブースティング(XGBRegressor)のパラメータ探索グリッドサーチ関数

In [ ]:
def run_xgb_regression_and_analysis_grid(train_df, test_df, var_dict, param_grid, eval_metric, scoring, save_path):
    """
    説明変数と目的変数を指定して、XGBoostingによる機械学習を実行し、予測値を計算する関数。
    :param df: データフレーム。説明変数と目的変数が含まれる。
    :type df: pandas.DataFrame
    :param var_dict: 説明変数と目的変数の辞書。キーが目的変数名で、値が説明変数名のリスト。
    :type var_dict: dict of str to list of str
    :param save_path: グラフを保存するフォルダのパス。
    :type save_path: str
    :return: 構築したモデルと予測値の辞書
    :rtype: tuple of dict
    """
    # 入力検証
    validate_input(var_dict,train_df)
    validate_input(var_dict,test_df)

    # 指定されたパスにフォルダがなければ作成
    create_folder(save_path)

    models = {}
    train_y_preds = {}
    test_y_preds = {}

    # 結果を格納するDataFrame作成
    result_df = pd.DataFrame(columns=['y_var', 'train_mse', 'train_mae', 'train_r2', 'test_mse', 'test_mae', 'test_r2'])

    for y_var, x_vars in var_dict.items():
        
        eval_set = [(train_df[x_vars], train_df[y_var])]
        # モデルインスタンス作成
        model = XGBRegressor(n_estimators=100, early_stopping_rounds=15, eval_set=eval_set, eval_metric=eval_metric)
        model.fit(train_df[x_vars], train_df[y_var],  verbose=False)

        # グリッドサーチ実行
        grid_search = GridSearchCV(estimator=model,
                                   param_grid=param_grid,
                                   cv=3,
                                   scoring=scoring,
                                   n_jobs=-1)
        grid_search.fit(train_df[x_vars], train_df[y_var], eval_set=eval_set, eval_metric=eval_metric, verbose=False)

        # 最適なパラメータで再学習
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        models[y_var] = best_model
        best_params_df = pd.DataFrame.from_dict(best_params, orient='index', columns=['value'])
        best_params_file_path = os.path.join(save_path, f'best_params_{y_var}.csv')
        best_params_df.to_csv(best_params_file_path)

        # 学習用データでの予測と評価指標計算
        train_y_pred = best_model.predict(train_df[x_vars])
        train_y_preds[y_var] = train_y_pred
        train_mse = mean_squared_error(train_df[y_var], train_y_pred)
        train_mae = mean_absolute_error(train_df[y_var], train_y_pred)
        train_r2 = r2_score(train_df[y_var], train_y_pred)
        
        # 検証用データでの予測と評価指標計算
        test_y_pred = best_model.predict(test_df[x_vars])
        test_y_preds[y_var] = test_y_pred
        test_mse = mean_squared_error(test_df[y_var], test_y_pred)
        test_mae = mean_absolute_error(test_df[y_var], test_y_pred)
        test_r2 = r2_score(test_df[y_var], test_y_pred)

        # 結果のサマリ表示
        """
        print(f'{y_var} の学習用MSE:', train_mse)
        print(f'{y_var} の学習用MAE:', train_mae)
        print(f'{y_var} の学習用決定係数:', train_r2)

        print(f'{y_var} の検証用MSE:', test_mse)
        print(f'{y_var} の検証用MAE:', test_mae)
        print(f'{y_var} の検証用決定係数:', test_r2)
        print('\n')
        """
        
        # 結果をDataFrameに追加
        result_df.loc[len(result_df)] = [y_var, train_mse, train_mae, train_r2, test_mse, test_mae, test_r2]

        # 残差計算
        residuals = train_df[y_var] - train_y_pred

        # 標準化残差計算
        standardized_residuals = (residuals - residuals.mean()) / residuals.std()


        # グラフのプロット部
        # 観測値、予測値 vs. time
        fig, axs = plt.subplots(2, 1, figsize=(50, 16))
        axs[0].plot(train_df['time'], train_df[y_var], label='Actual')
        axs[0].plot(train_df['time'], train_y_pred, label='Predicted')
        axs[0].set_title('Train Data')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel(y_var)
        axs[0].legend()
        
        axs[1].plot(test_df['time'], test_df[y_var], label='Actual')
        axs[1].plot(test_df['time'], test_y_pred, label='Predicted')
        axs[1].set_title('Test Data')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel(y_var)
        axs[1].legend()
        
        fig.suptitle(f'{y_var} - Actual vs Predicted')
        plt.savefig(os.path.join(save_path, f'{y_var}_yy_vs_time_{eval_metric}.png'))
        plt.close(fig)

        # 残差 vs. 説明変数
        fig, ax = plt.subplots(1, len(x_vars), figsize=(10*len(x_vars), 20))
        for i in range(len(x_vars)):
            if len(x_vars) > 1:
                ax[i].scatter(train_df[x_vars[i]], residuals)
                ax[i].set_xlabel(x_vars[i])
                ax[i].set_ylabel('Residuals')
            else:
                ax.scatter(train_df[x_vars[i]], residuals)
                ax.set_xlabel(x_vars[i])
                ax.set_ylabel('Residuals')
        plt.savefig(os.path.join(save_path,f'{y_var}_residuals_vs_xvars_{eval_metric}.png'))
        plt.close(fig)
        del train_y_pred, test_y_pred
        gc.collect()

    result_file_path = os.path.join(save_path, f'result_{eval_metric}.csv')
    result_df.to_csv(result_file_path,index=False)
    return models


In [ ]:
# データの読み込み
train_df = df.loc[(df.index >= '2022-12-1 00:00:00') & (df.index < '2023-2-1 00:00:00')].reset_index()
test_df = df.loc[(df.index >= '2022-11-01 00:00:00') & (df.index < '2022-12-1 00:00:00')].reset_index()

var_dict = {
    '3F':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '2F':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '1F':['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    'all':['ipa', 'sto', 'han', 'div', 'col', 'alt']}

# グリッドサーチ用パラメータ設定
param_grid = {
    'learning_ration': [0.01, 0.1, 0.2],
    'min_child_weight': [1, 2, 4, 6, 8],
    'max_depth': [2, 3, 4],
    'colsample_bytree': [0.2, 0.5, 0.8, 1.0],
    'subsample': [0.2, 0.6, 1.0]
    }

scoring = 'neg_mean_absolute_error'
eval_metric = 'mae'

save_path = "/workspaces/data/flow/xgboost_grid_param12-1"

# 時間(Hour)列を作成する
train_df = extract_hour(train_df)
test_df = extract_hour(test_df)

# 時間列に対してOne Hot Encodingを実行する
train_df = one_hot_encode_hour(train_df)
test_df = one_hot_encode_hour(test_df)

# 1から23までの整数のリストを作成
hours = list(range(1, 24))

# hour_1, hour_2, ..., hour_23 の形式のダミー変数名のリストを作成
hour_dummies = [f'hour_{hour}' for hour in hours]

# hourのダミー変数を追加した新しいvar_dict
updated_var_dict = {}
for target, features in var_dict.items():
    updated_features = features + hour_dummies
    updated_var_dict[target] = updated_features

models = run_xgb_regression_and_analysis_grid(train_df, test_df, updated_var_dict, param_grid, eval_metric, scoring, save_path)

#### 勾配ブースティング(XGBRegressor)の調整済みパラメータによるフィッティング関数と交差検証関数

In [ ]:
def train_and_analyze_xgb_regression(train_df, test_df, target_to_features, params_directory, eval_metric, scoring, save_directory):
    """
    XGBoost回帰モデルを学習し、予測と分析を行う関数。
    Args:
        train_df (pd.DataFrame): 学習用データセット。
        test_df (pd.DataFrame): 検証用データセット。
        target_to_features (dict): 目的変数をキー、説明変数のリストを値とした辞書。
        params_directory (str): ハイパーパラメータが保存されたディレクトリへのパス。
        eval_metric (str): 評価指標。
        scoring (str): スコアリング方法。
        save_directory (str): 結果を保存するディレクトリへのパス。
    Returns:
        dict: 学習済みモデルを格納した辞書。
    """

    # Validate inputs
    validate_input(target_to_features, train_df)
    validate_input(target_to_features, test_df)

    # Create the specified directory if it does not exist
    create_directory_if_not_exists(save_directory)

    models = {}
    train_y_preds = {}
    test_y_preds = {}

    # Create a DataFrame to store the results
    result_df = pd.DataFrame(columns=['y_var', 'train_mse', 'train_mae', 'train_r2', 'test_mse', 'test_mae', 'test_r2'])

    for target, features in target_to_features.items():
        
        eval_set = [(train_df[features], train_df[target])]

        # Load hyperparameters
        hyperparams = load_hyperparameters_from_csv(params_directory, target)

        # Create model parameters
        model_params = {
            'n_estimators': 100,
            'early_stopping_rounds': 15,
            'eval_set': eval_set,
            'eval_metric': eval_metric,
            'verbose': False
        }

        # Update model parameters with hyperparameters from the CSV file
        model_params.update(hyperparams)

        # Create a model instance
        model = XGBRegressor(**model_params)        
        model.fit(train_df[features], train_df[target], eval_set=eval_set)

        models[target] = model

        # Predict and calculate evaluation metrics for the training data
        train_y_pred = model.predict(train_df[features])
        train_y_preds[target] = train_y_pred
        train_mse = mean_squared_error(train_df[target], train_y_pred)
        train_mae = mean_absolute_error(train_df[target], train_y_pred)
        train_r2 = r2_score(train_df[target], train_y_pred)
        
        # Predict and calculate evaluation metrics for the test data
        test_y_pred = model.predict(test_df[features])
        test_y_preds[target] = test_y_pred
        test_mse = mean_squared_error(test_df[target], test_y_pred)
        test_mae = mean_absolute_error(test_df[target], test_y_pred)
        test_r2 = r2_score(test_df[target], test_y_pred)
        
        # Add the results to the DataFrame
        result_df.loc[len(result_df)] = [target, train_mse, train_mae, train_r2, test_mse, test_mae, test_r2]

        # Calculate residuals
        residuals = train_df[target] - train_y_pred

        # Calculate standardized residuals
        standardized_residuals = (residuals - residuals.mean()) / residuals.std()

        # Plot the graphs
        plot_actual_vs_predicted_and_residuals(train_df, test_df, target, features, train_y_pred, test_y_pred, residuals, save_directory, eval_metric)
        del train_y_pred, test_y_pred
        gc.collect()

    result_file_path = os.path.join(save_directory, f'result_{eval_metric}.csv')
    result_df.to_csv(result_file_path, index=False)
    pkl_file_path = os.path.join(save_directory, f'models_{eval_metric}.pkl')
    pickle_dump(models, pkl_file_path)
    return models

def perform_xgb_cross_validation(df, updated_target_to_features, params_directory, eval_metric, scoring, save_directory, n_splits):
    """
    クロスバリデーションによるXGBoost回帰モデルの学習と評価を行う関数。
    Args:
        df (pd.DataFrame): 全データセット。
        updated_target_to_features (dict): 目的変数をキー、説明変数のリストを値とした辞書。ダミー変数が更新されたもの。
        params_directory (str): ハイパーパラメータが保存されたディレクトリへのパス。
        eval_metric (str): 評価指標。
        scoring (str): スコアリング方法。
        save_directory (str): 結果を保存するディレクトリへのパス。
    Return:
        dict: 学習済みモデルを格納した辞書。
    """
    kf = KFold(n_splits)
    models = {}
    for i, (train_index, test_index) in enumerate(kf.split(df)):
        train_df = df.iloc[train_index]
        test_df = df.iloc[test_index]
        models.update(train_and_analyze_xgb_regression(train_df, test_df, updated_target_to_features, params_directory, eval_metric, scoring, f'{save_directory}/{i}'))
        pkl_file_path = os.path.join(save_directory, f'models_cv{n_splits}_{eval_metric}.pkl')
        pickle_dump(models, pkl_file_path)
    return models


In [ ]:
# Load the data
df_t = df.loc[(df.index >= '2022-11-1 00:00:00') & (df.index < '2023-3-30 00:00:00')].reset_index()

# scoring = 'neg_mean_absolute_error'
# eval_metric = 'mae'
# scoring = 'r2'
# eval_metric = 'rmse'
scoring = 'neg_mean_squared_error'
eval_metric = 'rmse'

n_sprits = 5

params_path = "/workspaces/data/flow/best_params/"
save_directory = f"/workspaces/data/flow/xgboost_crossvalidation_cv{n_sprits}_{eval_metric}"

# Create an 'hour' column
df_t = add_hour_column(df_t)

# Perform one-hot encoding on the 'hour' column
df_t = one_hot_encode_hours(df_t)

# Create a list of integers from 1 to 23
hours = list(range(1, 24))

# Create a list of dummy variable names in the format hour_1, hour_2, ..., hour_23
hour_dummies = [f'hour_{hour}' for hour in hours]

var_dict = {
    '3F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '2F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '1F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    'all': ['ipa', 'sto', 'han', 'div', 'col', 'alt']
}

# Create a new var_dict with the 'hour' dummy variables added
updated_var_dict = {}
for target, features in var_dict.items():
    updated_features = features + hour_dummies
    updated_var_dict[target] = updated_features

models = perform_xgb_cross_validation(df_t, updated_var_dict, params_path, eval_metric, scoring, save_directory, n_sprits)


#### 複数のモデルからベストなモデルを選定する関数

In [ ]:
def select_best_model(models_directory, df, target_to_features, scoring, save_directory):
    # 時間（Hour）列を作成
    df = add_hour_column(df)
    
    # 時間列に対してOne Hot Encodingを実行
    df = one_hot_encode_hours(df)

    # Create the specified directory if it does not exist
    create_directory_if_not_exists(save_directory)

    best_scores = {}
    best_models = {}
    best_model_paths = {}

    # Scoring functions
    scoring_functions = {
        'mse': (mean_squared_error, float('inf')),
        'mae': (mean_absolute_error, float('inf')),
        'r2': (r2_score, float('-inf')),
        'neg_mean_absolute_error': (mean_absolute_error, float('inf')),
    }

    if scoring not in scoring_functions:
        raise ValueError(f"Invalid scoring method '{scoring}'. Supported methods are {list(scoring_functions.keys())}")

    score_function, initial_score = scoring_functions[scoring]

    for model_file in os.listdir(models_directory):
        if model_file.endswith(".pkl"):
            file_path = os.path.join(models_directory, model_file)
            with open(file_path, 'rb') as file:
                models = pickle.load(file)
            for target, model in models.items():
                features = target_to_features[target]
                y_true = df[target]
                y_pred = model.predict(df[features])
                score = score_function(y_true, y_pred)
                # print(f"Model: {model_file}, Score: {score}")
                if target not in best_scores or (scoring == 'r2' and score > best_scores[target]) or (scoring == 'mae' and score < best_scores[target]):
                    best_scores[target] = score
                    best_models[target] = model
                    best_model_paths[target] = file_path
    pkl_file_path = os.path.join(save_directory, f'best_models_{scoring}.pkl')
    pickle_dump(best_models, pkl_file_path)
    return best_models, best_scores, best_model_paths

models_directory = '/workspaces/data/flow/models/'
result_directory = '/workspaces/data/flow/bestmodels/'

# Load the data
df_t = df.loc[(df.index >= '2022-11-6 00:00:00') & (df.index < '2023-1-1 00:00:00')].reset_index()

var_dict = {
    '3F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '2F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '1F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    'all': ['ipa', 'sto', 'han', 'div', 'col', 'alt']
}

# Create a list of integers from 1 to 23
hours = list(range(1, 24))

# Create a list of dummy variable names in the format hour_1, hour_2, ..., hour_23
hour_dummies = [f'hour_{hour}' for hour in hours]

# Create a new var_dict with the 'hour' dummy variables added
updated_var_dict = {}
for target, features in var_dict.items():
    updated_features = features + hour_dummies
    updated_var_dict[target] = updated_features

scoring_list = ['r2', 'mse', 'mae']

for scoring in scoring_list:
    _, result, path = select_best_model(models_directory, df_t, updated_var_dict, scoring, result_directory)
    print(scoring)
    print(result)
    print(path)

In [ ]:
models_directory = '/workspaces/data/flow/models/'

# Load the data
df_t = df.loc[(df.index >= '2022-11-6 00:00:00') & (df.index < '2023-1-1 00:00:00')].reset_index()

# Create a list of integers from 1 to 23
hours = list(range(1, 24))

# Create a list of dummy variable names in the format hour_1, hour_2, ..., hour_23
hour_dummies = [f'hour_{hour}' for hour in hours]

var_dict = {
    '3F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '2F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '1F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    'all': ['ipa', 'sto', 'han', 'div', 'col', 'alt']
}

# Create a new var_dict with the 'hour' dummy variables added
updated_var_dict = {}
for target, features in var_dict.items():
    updated_features = features + hour_dummies
    updated_var_dict[target] = updated_features

list = ['r2', 'mse', 'mae']

for scoring in list:
    _, result, path = select_best_model(models_directory, df_t, updated_var_dict, scoring)
    print(scoring)
    print(result)
    print(path)


In [ ]:
def print_evaluate_values_models(models_directory, df, target_to_features, save_directory):
    """
    構築済みのモデルを評価する関数。
    :param models_file_path: 各目的変数のモデルが格納された辞書型変数が格納されたpklファイルへのpath
    :type models_file_path: the path to pkl file of dict{key:str(target), value:model}
    :param df: 元データ
    :type df: pd.DataFrame
    :param target_to_features: 説明変数と目的変数の辞書。キーが目的変数名で、値が説明変数名のリスト。
    :type target_to_features: dict of str to list of str
    """
    # 時間（Hour）列を作成
    df = add_hour_column(df)
    
    # 時間列に対してOne Hot Encodingを実行
    df = one_hot_encode_hours(df)

    # Scoring functions
    scoring_functions = {
        'mse': (mean_squared_error, float('inf')),
        'mae': (mean_absolute_error, float('inf')),
        'r2': (r2_score, float('-inf')),
    }

    columns = ['model_file']
    for target in target_to_features.keys():
        for scoring in scoring_functions.keys():
            columns.append(f'{target}_{scoring}')
    scores_df = pd.DataFrame(columns=columns)

    # 各モデルファイルに対してスコアを計算し、DataFrameに追加します。
    for model_file in os.listdir(models_directory):
        if model_file.endswith(".pkl"):
            file_path = os.path.join(models_directory, model_file)
            models = pickle_load(file_path)
            row = {'model_file': model_file}
            for target, model in models.items():
                features = target_to_features[target]
                y_true = df[target]
                y_pred = model.predict(df[features])
                for scoring, scoring_function  in scoring_functions.items():
                    score = scoring_function[0](y_true, y_pred)
                    row[f'{target}_{scoring}'] = score
            scores_df = scores_df.append(row, ignore_index=True)
    scores_df.to_csv(f'{result_directory}result.csv')

models_directory = '/workspaces/data/flow/bestmodels/'
result_directory = '/workspaces/data/flow/result/'

# Load the data
df_t = df.loc[(df.index >= '2022-12-1 00:00:00') & (df.index < '2023-11-1 00:00:00')].reset_index()
result_df = pd.DataFrame()

# Create a list of integers from 1 to 23
hours = list(range(1, 24))

# Create a list of dummy variable names in the format hour_1, hour_2, ..., hour_23
hour_dummies = [f'hour_{hour}' for hour in hours]

var_dict = {
    '3F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '2F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    '1F': ['ipa', 'sto', 'han', 'div', 'col', 'alt'],
    'all': ['ipa', 'sto', 'han', 'div', 'col', 'alt']
}

# Create a new var_dict with the 'hour' dummy variables added
updated_var_dict = {}
for target, features in var_dict.items():
    updated_features = features + hour_dummies
    updated_var_dict[target] = updated_features

print_evaluate_values_models(models_directory, df_t, updated_var_dict, result_directory)
